<p style="font-weight:bold;"> <span style="font-size: 36px"> Map Template </span> </p>

This notebook exports the data nodes, data node states, and data node parameters belonging to a specific data partition into an excel file.
<br> This file contains reference data used to prepopulate dropdows and validations to facilitate editing.
<br> This can be modified and imported in the calculation engine to create and modify data.

After setting the desired target Partition, the Data present in the workspace is ready to be exported. 

The default location of the exported files is the project file storage. A **download** link is generated and available for as long as the session is active. 

# Select Initialization

Choose to run the Map Template either with the set of Systemorph data in memory or with the data present in the Database: 
- #!eval-notebook "../Database/Configure" : connects to a physical Database
- #!eval-notebook "../Import/CloseImportTemplate" : uses the in-memory set up

We use here the in-memory set up.

In [0]:
#!eval-notebook "../Import/CloseImportTemplate"

# Workspace Initialization 

Firstly, the Workspace is initialized with basic dimensions and with Data Nodes. For a list of the dimensions please refer to the DataModel/DataStructure notebook.

In [0]:
Workspace.InitializeFrom(DataSource);

# Map Template: Data Node

[DataNodes](https://portal.systemorph.cloud/project/ifrs17/env/v1.0.0/DataModel/DataStructure#data-node) defines the properties of [Portfolios](https://portal.systemorph.cloud/project/ifrs17/env/v1.0.0/DataModel/DataStructure#portfolios) and [Group of Contracts](https://portal.systemorph.cloud/project/ifrs17/env/v1.0.0/DataModel/DataStructure#group-of-contracts).

## Set Partition

In [0]:
static var partition = new PartitionByReportingNode() { ReportingNode = "CH" };
await Workspace.Partition.SetAsync<PartitionByReportingNode>( partition );

## Export

In [0]:
await Export.ToExcel("DataNodes")
    .WithSource(Workspace)
    .PortfolioConfiguration<ReinsurancePortfolio>()
    .PortfolioConfiguration<InsurancePortfolio>()
    .GroupofContractConfiguration<GroupOfReinsuranceContract>(typeof(ReinsurancePortfolio))
    .GroupofContractConfiguration<GroupOfInsuranceContract>(typeof(InsurancePortfolio))
    .MainTabConfigurationWoScenario(partition)
.ExecuteAsync()

# Map Template: Data Node State

[Data Node State](https://portal.systemorph.cloud/project/ifrs17/env/v1.0.0/DataModel/DataStructure#data-node-state) defines whether the instance is active (used in import/output) or inactive (present in the DataSource but not used in input/output operations).

## Set Partition

In [0]:
static var partition = new PartitionByReportingNodeAndPeriod() { ReportingNode = "CH", Year = 2020, Month = 12 };
await Workspace.Partition.SetAsync<PartitionByReportingNode>( new PartitionByReportingNode() { ReportingNode = partition.ReportingNode } );

## Export

The data is retrieved for the Workspace and prepared for export.

In [0]:
var dataNodeStates = (await Workspace.Query<DataNodeState>()
                        .Where(x => (x.Year == partition.Year && x.Month <= partition.Month) || x.Year < partition.Year).ToArrayAsync())
                        .GroupBy(x => x.DataNode)
                        .Select(x => x.OrderByDescending(y => y.Year).ThenByDescending(y => y.Month))
                        .Select(x => x.Last())
                        .ToArray();

Export is performed.

In [0]:
await Export.ToExcel("DataNodeState")
    .WithSource(Workspace)
    .StateEnumConfiguration() 
    .DataNodeStateConfiguration(dataNodeStates)
    .MainTabConfigurationWoScenario(partition)
.ExecuteAsync()

# Map Template: Data Node Parameter

[Data Node Parameters](https://portal.systemorph.cloud/project/ifrs17/env/v1.0.0/DataModel/DataStructure#data-node-parameters) are defined at the Group of Contract level and are used during the import calculation. 

## Set Partition

In [0]:
static var partition = new PartitionByReportingNodeAndPeriod() { ReportingNode = "CH", Year = 2020, Month = 12 };
await Workspace.Partition.SetAsync<PartitionByReportingNode>( new PartitionByReportingNode() { ReportingNode = partition.ReportingNode } );

## Export

The data is retrieved for the Workspace and prepared for export.

In [0]:
var dataNodeParameters = (await Workspace.Query<DataNodeParameter>()
                        .Where(x => (x.Year == partition.Year && x.Month <= partition.Month) || x.Year < partition.Year).ToArrayAsync())
                        .GroupBy(x => x.GetType().Name)
                        .ToDictionary(x => x.Key, 
                                      x => x.GroupBy(y => y.DataNode)
                                            .Select(y => y.OrderByDescending(z => z.Year).ThenByDescending(z => z.Month))
                                            .Select(y => y.Last())
                                            .ToArray() );

Export is performed.

In [0]:
await Export.ToExcel("DataNodeParameter")
    .WithSource(Workspace)
    .DataNodeParameterConfiguration(dataNodeParameters)
    .MainTabConfiguration(partition)
.ExecuteAsync()